Target column (Prof.Bell) is `image_links`. However, in column `microfiche_links` there are **scans** of photos and documents  

In order to download pictures to your computer, run all code in **Preprocessing** section and then go to section **Downloading** and choose whether you want to download images from `images_links` or scnas from `microfiche_links` by running relevant section (**Image downloading** or **Scan downloading**). Don't forget to specify amount of pictures to download `OBJECTS_TO_DOWNLOAD`

# Preprocessing

In [2]:
import pandas as pd
import requests
import re
import numpy as np
import os

In [3]:
file_name = 'DATASETS_DONT_TOUCH/dataset_CLEAN.csv'
df = pd.read_csv(file_name, sep=';', encoding='utf-8')

# print(df.shape)
# df.head()

Converting strings in coulmn image_links - microfiche_archive_links back into lists of strings

In [4]:
# Took ~20 seconds
 
columns_to_convert = df.columns[3:]

for i in range(df.shape[0]):
    for column in columns_to_convert:
        _ = df.iloc[i][column]
        if _ == '[]':
            df.at[i, column] = []
        else:
            df.at[i, column] =  _[2:-2].split('\', \'')


# Downloading

In [ ]:
sum_img, sum_scans, sum_dif = 0, 0, 0

for i in range(df.shape[0]):
    num_img, num_scans = len(df.iloc[i].image_links), len(df.iloc[i].microfiche_links)
    sum_img += num_img
    sum_scans += num_scans
    sum_dif += num_img == num_scans


print(f'Amount of objects in dataset: {df.shape[0]}')
print(f'Amount of images in \"image_links\": {sum_img}')
print(f'Amount of images in \"microfiche_links\": {sum_scans}')
print(f'Cases, when only scans or only images are given: {sum_dif}')

## Functions

In [7]:
def download_image(url, save_path):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Open a file in binary mode and write the image content
            with open(save_path, 'wb') as file:
                file.write(response.content)
            # print(f"Image successfully downloaded: {save_path}")
        else:
            print(f"Failed to retrieve image from {url}. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")

def create_folder(folder_name):
    # Check if the folder exists
    if not os.path.exists(folder_name):
        # Create the folder if it doesn't exist
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created.")
    else:
        print(f"Folder '{folder_name}' already exists.")

## Downloading

In [ ]:
# One object consists of many images
OBJECTS_TO_DOWNLOAD = 10

### Image downloading

In [8]:
# Upload images from image_links to your computer

create_folder('images')

for i in range(OBJECTS_TO_DOWNLOAD):
    j = 0
    for link in df.iloc[i].image_links:

        # Build a name for the image
        name = ''
        line = df.iloc[i].id_persistent
        if line != 'NO_ID_PERSISTENT':
            name += re.findall(r'.*/([^/]*)$', line)[0] + '-'
        else:
            name += 'NO_ID_PERSISTENT-'
        name += re.findall(r'.*/([^/]*)$', link)[0]

        save_path = f'images/img-{name}.jpg'
        download_image(link, save_path)
        j += 1


Folder 'images' already exists.


### Scan downloading

In [9]:
# Upload scans from microfiche_links to your computer

create_folder('scans')

# One object consists of many images

for i in range(OBJECTS_TO_DOWNLOAD):
    j = 0
    for link in df.iloc[i].microfiche_links:

        # Build a name for the image
        name = ''
        line = df.iloc[i].id_persistent
        if line != 'NO_ID_PERSISTENT':
            name += re.findall(r'.*/([^/]*)$', line)[0] + '-'
        else:
            name += 'NO_ID_PERSISTENT-'
        name += re.findall(r'.*/([^/]*)$', link)[0]

        save_path = f'scans/scn-{name}.jpg'
        download_image(link, save_path)
        j += 1


Folder 'scans' already exists.
